[Rendered Version of this notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/CPS%20Comparisons.ipynb)

In [1]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list, percentile
output_notebook()
import json

Loading BokehJS ...

### Missing Variables

In [2]:
cps = pd.read_csv('../taxdata/cps_data/cps_ben_full.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
e03230: Tuition and fees from Form 8917
e03400: Penalty on early withdrawal of sa

In [3]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [4]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2015)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2015)
pol_cps = Policy(start_year=2015, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
calc_cps.advance_to_year(2015)

In [6]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [7]:
create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,-182,"16,318,429",-542,"9,212,955","6,107","14,925",7,"4,916",26,1,-547,0,0,1,0,0,90,-90,160,70
1,"8,802","16,320,544","4,791","11,439,177","7,366","177,336",112,"5,630",115,8,"4,742","14,199",0,8,2,0,718,-711,626,-85
2,"15,277","16,319,424","8,267","11,894,865","7,536","957,115",706,"6,004","1,283",121,"7,915","34,552",1,122,22,0,983,-884,"1,094",211
3,"21,865","16,319,273","14,491","12,587,216","7,423","2,379,352","1,558","6,657","3,968",403,"13,596","31,097",2,405,68,0,"1,174",-837,"1,969","1,132"
4,"29,317","16,319,617","21,952","11,735,601","7,127","4,080,354","2,705","7,199","8,369",953,"20,325","31,232",3,956,135,0,"1,025",-205,"3,025","2,820"
5,"38,837","16,318,693","30,962","10,446,955","6,627","5,685,388","4,276","7,626","14,676","1,744","28,430","24,728",4,"1,748",227,0,600,921,"4,277","5,198"
6,"51,862","16,320,896","44,180","8,784,466","5,739","7,465,511","6,662","8,015","24,717","3,089","40,224","29,255",3,"3,092",348,0,164,"2,579","6,029","8,608"
7,"71,353","16,318,688","64,672","6,529,178","4,325","9,784,045","10,238","8,792","41,638","5,814","58,780","25,330",4,"5,818",495,0,24,"5,299","8,843","14,142"
8,"103,024","16,320,295","94,946","3,773,678","2,560","12,544,498","15,159","9,903","67,451","10,092","86,680","39,119",5,"10,097",595,0,4,"9,498","13,090","22,588"
9,"245,340","16,319,911","231,860","1,511,055","1,087","14,808,326","24,993","9,765","196,078","44,674","219,890","2,628,898",618,"45,292",122,374,0,"45,544","21,897","67,442"


### PUF Distribution Table

In [8]:
create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,953","16,441,745","-7,255","11,288,964","6,060","9,916",22,"4,171",52,5,"-7,636",0,0,5,0,7,144,-133,349,216
1,"8,726","16,442,575","6,124","13,702,321","7,176","46,911",25,"4,949",295,26,"6,103","12,231",0,26,0,4,790,-760,844,84
2,"15,492","16,442,473","10,161","13,277,515","7,736","283,102",186,"6,523","1,539",151,"10,032","27,238",1,152,20,7,"1,645","-1,507","1,438",-69
3,"22,631","16,442,367","15,600","14,440,049","7,933","705,376",574,"6,900","4,187",442,"15,239","31,321",2,444,86,13,"1,463","-1,092","2,089",997
4,"31,366","16,444,016","23,159","14,356,040","8,060","1,539,134","1,322","7,252","9,158","1,058","22,337","19,623",2,"1,060",232,17,"1,044",-199,"2,967","2,768"
5,"42,263","16,442,576","33,797","13,357,837","7,800","2,941,897","2,747","7,407","17,433","2,134","32,170","17,946",3,"2,137",378,27,510,"1,277","4,085","5,362"
6,"56,489","16,442,166","48,182","11,438,896","7,153","4,969,071","5,255","7,781","28,571","3,774","45,059","15,594",2,"3,776",512,43,150,"3,157","5,464","8,620"
7,"77,132","16,442,982","70,261","9,094,430","6,146","7,341,053","8,443","8,656","47,139","6,849","65,546","47,432",5,"6,854",680,62,78,"6,157","7,964","14,121"
8,"114,373","16,442,021","106,430","5,905,115","4,330","10,534,367","14,229","10,190","77,752","12,288","98,863","194,696",20,"12,308",856,101,96,"11,457","12,912","24,369"
9,"352,442","16,443,547","334,905","2,023,885","1,504","14,407,177","37,721","9,612","286,464","71,832","319,414","4,710,693","2,449","74,280","1,725","2,410",50,"74,915","22,559","97,474"


### Diagnostic Table Comparison

In [9]:
cps_diag = create_diagnostic_table(calc_cps)

In [10]:
puf_diag = create_diagnostic_table(calc_puf)

In [11]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['CPS'] = cps_diag[2015]
diag_data['PUF'] = puf_diag[2015]
diag_data['% Change'] = ((cps_diag[2015] / puf_diag[2015]) - 1) * 100

In [12]:
diag_data

,SOI,CPS,PUF,% Change
Returns (#m),148.6,163.2,164.4,-0.7
AGI ($b),"9,771.00","8,414.2","10,546.0",-20.2
Itemizers (#m),44,57.9,42.8,35.3
Itemized Deduction ($b),1206.7,"1,083.9","1,159.7",-6.5
Standard Deduction Filers (#m),117.4,87.9,108.9,-19.3
Standard Deduction ($b),876.2,769.5,951.0,-19.1
Personal Exemption ($b),1121.6,"1,110.2","1,137.0",-2.4
Taxable Income ($b),6997.9,"5,847.7","7,770.9",-24.7
Regular Tax ($b),NaN,"1,091.8","1,620.7",-32.6
AMT Income ($b),NaN,"7,834.1","9,983.0",-21.5


In [13]:
(diag_data['% Change']['Itemizers (#m)'] + diag_data['% Change']['Itemized Deduction ($b)'] +
 diag_data['% Change']['Standard Deduction Filers (#m)'] + diag_data['% Change']['Standard Deduction ($b)'])

-9.5359939025916702

In [14]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.g20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 335.95 ($b)
Interest Paid: 342.21 ($b)
Net Casualty or Theft Loss: 4.78 ($b)
-----------------
Total: 682.94 ($b)


### Income Levels

In [15]:
inc_dict = OrderedDict()
inc_dict['CPS'] = [] 
inc_dict['PUF'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_cps = (calc_cps.records.e00200 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(was_cps)
was_puf = (calc_puf.records.e00200 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(was_puf)
was_diff = (was_cps - was_puf)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was_puf) * 100)

int_cps = (calc_cps.records.e00300 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(int_cps)
int_puf = (calc_puf.records.e00300 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(int_puf)
int_diff = (int_cps - int_puf)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_puf) * 100)

odiv_cps = (calc_cps.records.e00600 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(odiv_cps)
odiv_puf = (calc_puf.records.e00600 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(odiv_puf)
odiv_diff = (odiv_cps - odiv_puf)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv_puf) * 100)

qdiv_cps = (calc_cps.records.e00650 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(qdiv_cps)
qdiv_puf = (calc_puf.records.e00650 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(qdiv_puf)
qdiv_diff = (qdiv_cps - qdiv_puf)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv_puf) * 100)

biz_cps = (calc_cps.records.e00900 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(biz_cps)
biz_puf = (calc_puf.records.e00900 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(biz_puf)
biz_diff = (biz_cps - biz_puf)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz_puf) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,CPS,PUF,Diff,Pct Diff
WAS,"6,648,838,107,939.5","7,146,623,659,349.3","-497,785,551,409.8",-7.0
Taxable Interest,"91,960,641,825.4","98,016,622,270.4","-6,055,980,445.1",-6.2
Ordinary Dividends,"245,424,347,487.0","287,599,002,040.1","-42,174,654,553.1",-14.7
Qualified Dividends,"185,442,636,961.2","210,639,585,110.9","-25,196,948,149.7",-12.0
Business Income,"309,555,316,495.2","324,061,778,926.4","-14,506,462,431.3",-4.5


### Distribution of Income Variables

In [16]:
# Generate data for distribution plots
cps_dist = pd.DataFrame()
puf_dist = pd.DataFrame()
cps_was = distribution(calc_cps.records.e00200, calc_cps.records.s006, calc_cps.records.c00100)
puf_was = distribution(calc_puf.records.e00200, calc_puf.records.s006, calc_puf.records.c00100)
cps_int = distribution(calc_cps.records.e00300, calc_cps.records.s006, calc_cps.records.c00100)
puf_int = distribution(calc_puf.records.e00300, calc_puf.records.s006, calc_puf.records.c00100)
cps_odiv = distribution(calc_cps.records.e00600, calc_cps.records.s006, calc_cps.records.c00100)
puf_odiv = distribution(calc_puf.records.e00600, calc_puf.records.s006, calc_puf.records.c00100)
cps_qdiv = distribution(calc_cps.records.e00650, calc_cps.records.s006, calc_cps.records.c00100)
puf_qdiv = distribution(calc_puf.records.e00650, calc_puf.records.s006, calc_puf.records.c00100)
cps_biz = distribution(calc_cps.records.e00900, calc_cps.records.s006, calc_cps.records.c00100)
puf_biz = distribution(calc_puf.records.e00900, calc_puf.records.s006, calc_puf.records.c00100)
cps_dist['WAS'] = cps_was[1]
puf_dist['WAS'] = puf_was[1]
cps_dist['INT'] = cps_int[1]
puf_dist['INT'] = puf_int[1]
cps_dist['ODIV'] = cps_odiv[1]
puf_dist['ODIV'] = puf_odiv[1]
cps_dist['QDIV'] = cps_qdiv[1]
puf_dist['QDIV'] = puf_qdiv[1]
cps_dist['BIZ'] = cps_biz[1]
puf_dist['BIZ'] = puf_biz[1]
cps_dist['AGI Bin'] = index_list()
puf_dist['AGI Bin'] = index_list()
cps_dist['label'] = 'CPS'
puf_dist['label'] = 'PUF'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([cps_dist, puf_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [17]:
show(column(scatter_list))

In [18]:
cps_tot = pd.DataFrame()
puf_tot = pd.DataFrame()
cps_tot['WAS'] = cps_was[0]
puf_tot['WAS'] = puf_was[0]
cps_tot['INT'] = cps_int[0]
puf_tot['INT'] = puf_int[0]
cps_tot['ODIV'] = cps_odiv[0]
puf_tot['ODIV'] = puf_odiv[0]
cps_tot['QDIV'] = cps_qdiv[0]
puf_tot['QDIV'] = puf_qdiv[0]
cps_tot['BIZ'] = cps_biz[0]
puf_tot['BIZ'] = puf_biz[0]
cps_tot['AGI Bin'] = index_list()
puf_tot['AGI Bin'] = index_list()
cps_tot['label'] = 'CPS'
puf_tot['label'] = 'PUF'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([cps_tot, puf_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [19]:
show(column(total_list))

### Itemized Deduction Amounts

In [20]:
deductions_cps = {'Medical Expenses': (calc_cps.records.e17500[calc_cps.records.c04470 > 0] *
                                       calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_cps.records.e18400[calc_cps.records.c04470 > 0] *
                                             calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_cps.records.e18500[calc_cps.records.c04470 > 0] *
                                         calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_cps.records.e19200[calc_cps.records.c04470 > 0] *
                                     calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_cps.records.e19800[calc_cps.records.c04470 > 0] *
                                                    calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_cps.records.e20100[calc_cps.records.c04470 > 0] *
                                                        calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_cps.records.e20400[calc_cps.records.c04470 > 0] *
                                            calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_cps.records.g20500[calc_cps.records.c04470 > 0] *
                                           calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum()}
ded_cps_df = pd.DataFrame.from_dict(deductions_cps, 'index')
ded_cps_df.columns = ['Total']
ded_cps_df['source'] = 'CPS'

deductions_puf = {'Medical Expenses': (calc_puf.records.e17500[calc_puf.records.c04470 > 0] *
                                       calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_puf.records.e18400[calc_puf.records.c04470 > 0] *
                                             calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_puf.records.e18500[calc_puf.records.c04470 > 0] *
                                         calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_puf.records.e19800[calc_puf.records.c04470 > 0] *
                                                    calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_puf.records.e20100[calc_puf.records.c04470 > 0] *
                                                        calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_puf.records.e20400[calc_puf.records.c04470 > 0] *
                                            calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_puf.records.g20500[calc_puf.records.c04470 > 0] *
                                           calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()}
ded_puf_df = pd.DataFrame.from_dict(deductions_puf, 'index')
ded_puf_df.columns = ['Total']
ded_puf_df['source'] = 'PUF'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_cps_df, ded_puf_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [21]:
(calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()

318302533899.28314

In [22]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height'),
                                                            ('Data', '@source')])
show(ded_bar)

In [23]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference - CPS'] = ded_cps_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - CPS'] = 100 * ded_error_df['Difference - CPS'] / soi_deductions['Total']
ded_error_df['Difference - PUF'] = ded_puf_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - PUF'] = 100 * ded_error_df['Difference - PUF'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference - CPS,% Difference - CPS,Difference - PUF,% Difference - PUF
Charitable Cash Contributions,"6,382,229,509.0",4.1,"2,671,633,335.8",1.7
Charitable Non-Cash Contributions,"-29,836,512,087.3",-45.7,"-29,943,383,704.6",-45.8
Interest Paid,"9,796,287,376.5",3.2,"10,340,148,899.3",3.4
Medical Expenses,"-16,108,893,366.9",-12.5,"-5,985,934,555.6",-4.6
Net Casualty or Loss,"-2,204,349,000.0",-100.0,"2,411,947,393.6",109.4
Real Estate Taxes,"-57,509,601,924.6",-31.8,"27,226,854,392.2",15.0
State and Local Taxes,"-19,938,190,726.9",-6.1,"-1,491,588,810.8",-0.5
Total Misc. Expenses,"-22,006,233,592.0",-17.4,"10,319,333,533.9",8.2


### Refundable Credits

In [24]:
eitc_cps = (calc_cps.records.eitc * calc_cps.records.s006).sum()
c11070_cps = (calc_cps.records.c11070 * calc_cps.records.s006).sum()
c10960_cps = (calc_cps.records.c10960 * calc_cps.records.s006).sum()
personal_credit_cps = (calc_cps.records.personal_credit * calc_cps.records.s006).sum()
ctc_new_cps = (calc_cps.records.ctc_new * calc_cps.records.s006).sum()

eitc_puf = (calc_puf.records.eitc * calc_puf.records.s006).sum()
c11070_puf = (calc_puf.records.c11070 * calc_puf.records.s006).sum()
c10960_puf = (calc_puf.records.c10960 * calc_puf.records.s006).sum()
personal_credit_puf = (calc_puf.records.personal_credit * calc_puf.records.s006).sum()
ctc_new_puf = (calc_puf.records.ctc_new * calc_puf.records.s006).sum()

In [25]:
print 'Refundable Credits'
pd.DataFrame(OrderedDict({'PUF': [eitc_puf, c11070_puf, c10960_puf, personal_credit_puf, ctc_new_puf],
                          'CPS': [eitc_cps, c11070_cps, c10960_cps, personal_credit_cps, ctc_new_cps],
                          'Diff': [eitc_cps - eitc_puf, c11070_cps - c11070_puf, c10960_cps - c10960_puf,
                                   personal_credit_cps - personal_credit_puf,
                                   ctc_new_cps - ctc_new_puf]}),
                          index=['eitc', 'c11070', 'c10960', 'personal credit', 'ctc new'])

Refundable Credits


,Diff,PUF,CPS
eitc,"-7,363,363,471.2","65,703,321,662.4","58,339,958,191.2"
c11070,"-4,143,797,601.1","23,856,185,234.2","19,712,387,633.1"
c10960,"-8,627,081,579.1","8,627,081,579.1",0.0
personal credit,0.0,0.0,0.0
ctc new,0.0,0.0,0.0


In [26]:
eic0cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 0])
eic1cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 1])
eic2cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 2])
eic3cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 3])
eic0puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 0])
eic1puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 1])
eic2puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 2])
eic3puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 3]) 
eic_tot_cps = calc_cps.records.s006.sum()
eic_tot_puf = calc_puf.records.s006.sum()
print 'Percent and Number of Tax Units with Specified Number of EIC Qualified Children'
pd.DataFrame({'CPS': [eic0cps, eic1cps, eic2cps, eic3cps],
              'PUF': [eic0puf, eic1puf, eic2puf, eic3puf],
              '% - CPS': [eic0cps / eic_tot_cps, eic1cps / eic_tot_cps,
                          eic2cps / eic_tot_cps, eic3cps / eic_tot_cps],
              '% - PUF': [eic0puf / eic_tot_puf, eic1puf / eic_tot_puf,
                          eic2puf / eic_tot_puf, eic3puf / eic_tot_puf]})

Percent and Number of Tax Units with Specified Number of EIC Qualified Children


,% - CPS,% - PUF,CPS,PUF
0,0.7,0.9,"114,745,569.5","143,002,628.3"
1,0.1,0.1,"22,983,420.2","10,220,299.5"
2,0.1,0.0,"16,819,448.2","7,952,108.4"
3,0.1,0.0,"8,647,332.5","3,251,431.7"


### Benefit Programs

In [27]:
# Columns used for participation rates
cps['ssi_part'] = np.where(cps.ssi > 0, 1, 0)
cps['snap_part'] = np.where(cps.snap > 0, 1, 0)
cps['mcare_part'] = np.where(cps.mcare > 0, 1, 0)
cps['mcaid_part'] = np.where(cps.mcaid > 0, 1, 0)
cps['ss_part'] = np.where(cps.ss > 0, 1, 0)
cps['vb_part'] = np.where(cps.vb > 0, 1, 0)

In [28]:
benefits = pd.DataFrame({'SSI': [(cps.ssi * cps.s006).sum()],
                         'SNAP': [(cps.snap * cps.s006).sum()],
                         'Medicare': [(cps.mcare * cps.s006).sum()],
                         'Medicaid': [(cps.mcaid * cps.s006).sum()],
                         'Social Security': [(cps.ss * cps.s006).sum()],
                         'VB': [(cps.vb * cps.s006).sum()]}).transpose()
benefits.columns = ['Total']
print 'Benefits Totals'
benefits

Benefits Totals


,Total
Medicaid,"352,985,014,221.6"
Medicare,"488,236,708,639.1"
SNAP,"82,336,140,621.7"
SSI,"54,186,915,454.5"
Social Security,"630,068,059,192.4"
VB,"146,836,134,085.4"


#### Benefit Program Participation Rates

In [29]:
benefit_participation_names = [('ssi_part', 'SSI'), ('snap_part', 'SNAP'), ('mcare_part', 'Medicare'),
                               ('mcaid_part', 'Medicaid'), ('ss_part', 'Social Security'),
                               ('vb_part', "Veteran's Benefits")]

In [30]:
# Create list of bokeh figure displaying participation rates in each program
fig_list = list()
for item in benefit_participation_names:
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    f = figure(title='{} Participation Rate'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [31]:
show(column(fig_list))

#### Average Benefits Received by Participants

In [32]:
benefit_program_names = [('ssi', 'SSI'), ('snap', 'SNAP'), ('mcare', 'Medicare'),
                         ('mcaid', 'Medicaid'), ('ss', 'Social Security'),
                         ('vb', "Veteran's Benefits")]

In [33]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    bcps = cps[cps[item[0]] > 0]
    wcps = percentile(bcps, item[0], 100, 'e00200', 's006')
    f = figure(title='Average {} Benefit - Participants'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

taxcalc/utils.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pdf.sort_values(by=income_measure, inplace=True)
taxcalc/utils.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pdf['cumsum_temp'] = np.cumsum(pdf['s006'].values)
taxcalc/utils.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pdf['bins'] = pd.cut(pdf['cumsum_temp'], bins=bin_edges, labels=labels)
ta

In [34]:
show(column(fig_list))

#### Average Benefits Received by Entire Population

In [35]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    f = figure(title='Average {} Benefit - Entire Population'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [36]:
show(column(fig_list))